### ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Trabajar con arrays

#### array_zip

##### Ejemplo 1

In [0]:
array_data = [
    ("John",4,1),
    ("John",6,2),
    ("David",7,3),
    ("Mike",3,4),
    ("David",5,2),
    ("John",7,3),
    ("John",9,7),
    ("David",1,8),
    ("David",4,9),
    ("David",7,4),
    ("Mike",8,5),
    ("Mike",5,2),
    ("Mike",3,8),
    ("John",2,7),
    ("David",1,9)
]
array_schema = ["Name", "Score_1","Score_2"]

arrayDF = spark.createDataFrame(data=array_data, schema=array_schema)

display(arrayDF)

Name,Score_1,Score_2
John,4,1
John,6,2
David,7,3
Mike,3,4
David,5,2
John,7,3
John,9,7
David,1,8
David,4,9
David,7,4


In [0]:
from pyspark.sql import functions as F

masterDF = arrayDF.groupBy("Name").agg(F.collect_list('Score_1').alias('Array_Score_1'),F.collect_list('Score_2').alias('Array_Score_2'))

display(masterDF)

masterDF.printSchema()

Name,Array_Score_1,Array_Score_2
John,"List(4, 6, 7, 9, 2)","List(1, 2, 3, 7, 7)"
David,"List(7, 5, 1, 4, 7, 1)","List(3, 2, 8, 9, 4, 9)"
Mike,"List(3, 8, 5, 3)","List(4, 5, 2, 8)"


root
 |-- Name: string (nullable = true)
 |-- Array_Score_1: array (nullable = false)
 |    |-- element: long (containsNull = false)
 |-- Array_Score_2: array (nullable = false)
 |    |-- element: long (containsNull = false)



In [0]:
arr_zip_df = masterDF.withColumn("Zipped_value",F.arrays_zip("Array_Score_1","Array_Score_2"))

display(arr_zip_df)

arr_zip_df.show(10,False)

Name,Array_Score_1,Array_Score_2,Zipped_value
John,"List(4, 6, 7, 9, 2)","List(1, 2, 3, 7, 7)","List(List(4, 1), List(6, 2), List(7, 3), List(9, 7), List(2, 7))"
David,"List(7, 5, 1, 4, 7, 1)","List(3, 2, 8, 9, 4, 9)","List(List(7, 3), List(5, 2), List(1, 8), List(4, 9), List(7, 4), List(1, 9))"
Mike,"List(3, 8, 5, 3)","List(4, 5, 2, 8)","List(List(3, 4), List(8, 5), List(5, 2), List(3, 8))"


+-----+------------------+------------------+------------------------------------------------+
|Name |Array_Score_1     |Array_Score_2     |Zipped_value                                    |
+-----+------------------+------------------+------------------------------------------------+
|John |[4, 6, 7, 9, 2]   |[1, 2, 3, 7, 7]   |[{4, 1}, {6, 2}, {7, 3}, {9, 7}, {2, 7}]        |
|David|[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|[{7, 3}, {5, 2}, {1, 8}, {4, 9}, {7, 4}, {1, 9}]|
|Mike |[3, 8, 5, 3]      |[4, 5, 2, 8]      |[{3, 4}, {8, 5}, {5, 2}, {3, 8}]                |
+-----+------------------+------------------+------------------------------------------------+



##### Ejemplo 2

In [0]:
empDF = [
         ('Sales_dept',[{'emp_name':'John','salary':'1000','yrs_of_service':'10','Age':'33'},
                        {'emp_name':'David','salary':'2000','yrs_of_service':'15','Age':'40'},
                        {'emp_name':'Nancy','salary':'8000','yrs_of_service':'20','Age':'45'},
                        {'emp_name':'Mike','salary':'3000','yrs_of_service':'6','Age':'30'},
                        {'emp_name':'Rosy','salary':'6000','yrs_of_service':'8','Age':'32'}]),
         ('HR_dept',   [{'emp_name':'Edwin','salary':'6000','yrs_of_service':'8','Age':'31'},
                        {'emp_name':'Tomas','salary':'3000','yrs_of_service':'4','Age':'26'},
                        {'emp_name':'Sarah','salary':'12000','yrs_of_service':'22','Age':'49'},
                        {'emp_name':'Stella','salary':'15000','yrs_of_service':'25','Age':'52'},
                        {'emp_name':'Kevin','salary':'4000','yrs_of_service':'5','Age':'27'}
                       ])]
          
df_brand = spark.createDataFrame(data=empDF, schema=['Department', 'Employee'])
df_brand.printSchema()
display(df_brand)

root
 |-- Department: string (nullable = true)
 |-- Employee: array (nullable = true)
 |    |-- element: map (containsNull = true)
 |    |    |-- key: string
 |    |    |-- value: string (valueContainsNull = true)



Department,Employee
Sales_dept,"List(Map(emp_name -> John, yrs_of_service -> 10, salary -> 1000, Age -> 33), Map(emp_name -> David, yrs_of_service -> 15, salary -> 2000, Age -> 40), Map(emp_name -> Nancy, yrs_of_service -> 20, salary -> 8000, Age -> 45), Map(emp_name -> Mike, yrs_of_service -> 6, salary -> 3000, Age -> 30), Map(emp_name -> Rosy, yrs_of_service -> 8, salary -> 6000, Age -> 32))"
HR_dept,"List(Map(emp_name -> Edwin, yrs_of_service -> 8, salary -> 6000, Age -> 31), Map(emp_name -> Tomas, yrs_of_service -> 4, salary -> 3000, Age -> 26), Map(emp_name -> Sarah, yrs_of_service -> 22, salary -> 12000, Age -> 49), Map(emp_name -> Stella, yrs_of_service -> 25, salary -> 15000, Age -> 52), Map(emp_name -> Kevin, yrs_of_service -> 5, salary -> 4000, Age -> 27))"


In [0]:
df_brandZip = df_brand.withColumn("Zip",F.arrays_zip(df_brand["Employee"]))
display(df_brandZip)

Department,Employee,Zip
Sales_dept,"List(Map(emp_name -> John, yrs_of_service -> 10, salary -> 1000, Age -> 33), Map(emp_name -> David, yrs_of_service -> 15, salary -> 2000, Age -> 40), Map(emp_name -> Nancy, yrs_of_service -> 20, salary -> 8000, Age -> 45), Map(emp_name -> Mike, yrs_of_service -> 6, salary -> 3000, Age -> 30), Map(emp_name -> Rosy, yrs_of_service -> 8, salary -> 6000, Age -> 32))","List(List(Map(emp_name -> John, yrs_of_service -> 10, salary -> 1000, Age -> 33)), List(Map(emp_name -> David, yrs_of_service -> 15, salary -> 2000, Age -> 40)), List(Map(emp_name -> Nancy, yrs_of_service -> 20, salary -> 8000, Age -> 45)), List(Map(emp_name -> Mike, yrs_of_service -> 6, salary -> 3000, Age -> 30)), List(Map(emp_name -> Rosy, yrs_of_service -> 8, salary -> 6000, Age -> 32)))"
HR_dept,"List(Map(emp_name -> Edwin, yrs_of_service -> 8, salary -> 6000, Age -> 31), Map(emp_name -> Tomas, yrs_of_service -> 4, salary -> 3000, Age -> 26), Map(emp_name -> Sarah, yrs_of_service -> 22, salary -> 12000, Age -> 49), Map(emp_name -> Stella, yrs_of_service -> 25, salary -> 15000, Age -> 52), Map(emp_name -> Kevin, yrs_of_service -> 5, salary -> 4000, Age -> 27))","List(List(Map(emp_name -> Edwin, yrs_of_service -> 8, salary -> 6000, Age -> 31)), List(Map(emp_name -> Tomas, yrs_of_service -> 4, salary -> 3000, Age -> 26)), List(Map(emp_name -> Sarah, yrs_of_service -> 22, salary -> 12000, Age -> 49)), List(Map(emp_name -> Stella, yrs_of_service -> 25, salary -> 15000, Age -> 52)), List(Map(emp_name -> Kevin, yrs_of_service -> 5, salary -> 4000, Age -> 27)))"


In [0]:
df_brand_exp = df_brandZip.withColumn('Explode',F.explode(df_brandZip.Zip))
display(df_brand_exp)

Department,Employee,Zip,Explode
Sales_dept,"List(Map(emp_name -> John, yrs_of_service -> 10, salary -> 1000, Age -> 33), Map(emp_name -> David, yrs_of_service -> 15, salary -> 2000, Age -> 40), Map(emp_name -> Nancy, yrs_of_service -> 20, salary -> 8000, Age -> 45), Map(emp_name -> Mike, yrs_of_service -> 6, salary -> 3000, Age -> 30), Map(emp_name -> Rosy, yrs_of_service -> 8, salary -> 6000, Age -> 32))","List(List(Map(emp_name -> John, yrs_of_service -> 10, salary -> 1000, Age -> 33)), List(Map(emp_name -> David, yrs_of_service -> 15, salary -> 2000, Age -> 40)), List(Map(emp_name -> Nancy, yrs_of_service -> 20, salary -> 8000, Age -> 45)), List(Map(emp_name -> Mike, yrs_of_service -> 6, salary -> 3000, Age -> 30)), List(Map(emp_name -> Rosy, yrs_of_service -> 8, salary -> 6000, Age -> 32)))","List(Map(emp_name -> John, yrs_of_service -> 10, salary -> 1000, Age -> 33))"
Sales_dept,"List(Map(emp_name -> John, yrs_of_service -> 10, salary -> 1000, Age -> 33), Map(emp_name -> David, yrs_of_service -> 15, salary -> 2000, Age -> 40), Map(emp_name -> Nancy, yrs_of_service -> 20, salary -> 8000, Age -> 45), Map(emp_name -> Mike, yrs_of_service -> 6, salary -> 3000, Age -> 30), Map(emp_name -> Rosy, yrs_of_service -> 8, salary -> 6000, Age -> 32))","List(List(Map(emp_name -> John, yrs_of_service -> 10, salary -> 1000, Age -> 33)), List(Map(emp_name -> David, yrs_of_service -> 15, salary -> 2000, Age -> 40)), List(Map(emp_name -> Nancy, yrs_of_service -> 20, salary -> 8000, Age -> 45)), List(Map(emp_name -> Mike, yrs_of_service -> 6, salary -> 3000, Age -> 30)), List(Map(emp_name -> Rosy, yrs_of_service -> 8, salary -> 6000, Age -> 32)))","List(Map(emp_name -> David, yrs_of_service -> 15, salary -> 2000, Age -> 40))"
Sales_dept,"List(Map(emp_name -> John, yrs_of_service -> 10, salary -> 1000, Age -> 33), Map(emp_name -> David, yrs_of_service -> 15, salary -> 2000, Age -> 40), Map(emp_name -> Nancy, yrs_of_service -> 20, salary -> 8000, Age -> 45), Map(emp_name -> Mike, yrs_of_service -> 6, salary -> 3000, Age -> 30), Map(emp_name -> Rosy, yrs_of_service -> 8, salary -> 6000, Age -> 32))","List(List(Map(emp_name -> John, yrs_of_service -> 10, salary -> 1000, Age -> 33)), List(Map(emp_name -> David, yrs_of_service -> 15, salary -> 2000, Age -> 40)), List(Map(emp_name -> Nancy, yrs_of_service -> 20, salary -> 8000, Age -> 45)), List(Map(emp_name -> Mike, yrs_of_service -> 6, salary -> 3000, Age -> 30)), List(Map(emp_name -> Rosy, yrs_of_service -> 8, salary -> 6000, Age -> 32)))","List(Map(emp_name -> Nancy, yrs_of_service -> 20, salary -> 8000, Age -> 45))"
Sales_dept,"List(Map(emp_name -> John, yrs_of_service -> 10, salary -> 1000, Age -> 33), Map(emp_name -> David, yrs_of_service -> 15, salary -> 2000, Age -> 40), Map(emp_name -> Nancy, yrs_of_service -> 20, salary -> 8000, Age -> 45), Map(emp_name -> Mike, yrs_of_service -> 6, salary -> 3000, Age -> 30), Map(emp_name -> Rosy, yrs_of_service -> 8, salary -> 6000, Age -> 32))","List(List(Map(emp_name -> John, yrs_of_service -> 10, salary -> 1000, Age -> 33)), List(Map(emp_name -> David, yrs_of_service -> 15, salary -> 2000, Age -> 40)), List(Map(emp_name -> Nancy, yrs_of_service -> 20, salary -> 8000, Age -> 45)), List(Map(emp_name -> Mike, yrs_of_service -> 6, salary -> 3000, Age -> 30)), List(Map(emp_name -> Rosy, yrs_of_service -> 8, salary -> 6000, Age -> 32)))","List(Map(emp_name -> Mike, yrs_of_service -> 6, salary -> 3000, Age -> 30))"
Sales_dept,"List(Map(emp_name -> John, yrs_of_service -> 10, salary -> 1000, Age -> 33), Map(emp_name -> David, yrs_of_service -> 15, salary -> 2000, Age -> 40), Map(emp_name -> Nancy, yrs_of_service -> 20, salary -> 8000, Age -> 45), Map(emp_name -> Mike, yrs_of_service -> 6, salary -> 3000, Age -> 30), Map(emp_name -> Rosy, yrs_of_service -> 8, salary -> 6000, Age -> 32))","List(List(Map(emp_name -> John, yrs_of_service -> 10, salary -> 1000, Age -> 33)), List(Map(emp_name -> David, yrs_of_service -> 15, salary -> 2000, Age -> 40)), List(Map(e

In [0]:
df_brand_output = df_brand_exp.withColumn("employee_emp_name",df_brand_exp['Explode.Employee.emp_name'])\
                              .withColumn("employee_yrs_of_service",df_brand_exp['Explode.Employee.yrs_of_service'])\
                              .withColumn("employee_salary",df_brand_exp['Explode.Employee.salary'])\
                              .withColumn("employee_age",df_brand_exp['Explode.Employee.Age'])\
                              .drop("Explode")\
                              .drop("Zip")\
                              .drop("Employee")
display(df_brand_output)

Department,employee_emp_name,employee_yrs_of_service,employee_salary,employee_age
Sales_dept,John,10,1000,33
Sales_dept,David,15,2000,40
Sales_dept,Nancy,20,8000,45
Sales_dept,Mike,6,3000,30
Sales_dept,Rosy,8,6000,32
HR_dept,Edwin,8,6000,31
HR_dept,Tomas,4,3000,26
HR_dept,Sarah,22,12000,49
HR_dept,Stella,25,15000,52
HR_dept,Kevin,5,4000,27


#### array_intersect

In [0]:
empDF = [('John',[4,6,7,9,2],[1,2,3,7,7]),
         ('David',[7,5,1,4,7,1],[3,2,8,9,4,9]),
         ('Mike',[3,9,1,6,2],[1,2,3,5,8])   
        ]
df = spark.createDataFrame(data=empDF, schema=['Name','Array_1','Array_2'])
df.show()

+-----+------------------+------------------+
| Name|           Array_1|           Array_2|
+-----+------------------+------------------+
| John|   [4, 6, 7, 9, 2]|   [1, 2, 3, 7, 7]|
|David|[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|
| Mike|   [3, 9, 1, 6, 2]|   [1, 2, 3, 5, 8]|
+-----+------------------+------------------+



In [0]:
from pyspark.sql import functions as F

df_intersect = df.withColumn("Intersect",F.array_intersect(df["Array_1"],df["Array_2"]))
df_intersect.show()

+-----+------------------+------------------+---------+
| Name|           Array_1|           Array_2|Intersect|
+-----+------------------+------------------+---------+
| John|   [4, 6, 7, 9, 2]|   [1, 2, 3, 7, 7]|   [7, 2]|
|David|[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|      [4]|
| Mike|   [3, 9, 1, 6, 2]|   [1, 2, 3, 5, 8]|[3, 1, 2]|
+-----+------------------+------------------+---------+



#### array_except

In [0]:
empDF = [('John',[4,6,7,9,2],[1,2,3,7,7]),
         ('David',[7,5,1,4,7,1],[3,2,8,9,4,9]),
         ('Mike',[3,9,1,6,2],[1,2,3,5,8])   
        ]
df = spark.createDataFrame(data=empDF, schema=['Name','Array_1','Array_2'])
df.show()

+-----+------------------+------------------+
| Name|           Array_1|           Array_2|
+-----+------------------+------------------+
| John|   [4, 6, 7, 9, 2]|   [1, 2, 3, 7, 7]|
|David|[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|
| Mike|   [3, 9, 1, 6, 2]|   [1, 2, 3, 5, 8]|
+-----+------------------+------------------+



In [0]:
from pyspark.sql import functions as F

df_intersect = df.withColumn("Intersect",F.array_except(df["Array_1"],df["Array_2"]))
df_intersect.show()

+-----+------------------+------------------+---------+
| Name|           Array_1|           Array_2|Intersect|
+-----+------------------+------------------+---------+
| John|   [4, 6, 7, 9, 2]|   [1, 2, 3, 7, 7]|[4, 6, 9]|
|David|[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|[7, 5, 1]|
| Mike|   [3, 9, 1, 6, 2]|   [1, 2, 3, 5, 8]|   [9, 6]|
+-----+------------------+------------------+---------+



#### array_sort

In [0]:
empDF = [('John',[4,6,7,9,2]),
         ('David',[7,5,1,4,7,1]),
         ('Mike',[3,9,1,6,2])   
        ]
df = spark.createDataFrame(data=empDF, schema=['Name','Array_col'])
df.show()

+-----+------------------+
| Name|         Array_col|
+-----+------------------+
| John|   [4, 6, 7, 9, 2]|
|David|[7, 5, 1, 4, 7, 1]|
| Mike|   [3, 9, 1, 6, 2]|
+-----+------------------+



In [0]:
from pyspark.sql import functions as F

df_sort = df.withColumn("Sorted",F.array_sort(df["Array_col"]))
df_sort.show()

+-----+------------------+------------------+
| Name|         Array_col|            Sorted|
+-----+------------------+------------------+
| John|   [4, 6, 7, 9, 2]|   [2, 4, 6, 7, 9]|
|David|[7, 5, 1, 4, 7, 1]|[1, 1, 4, 5, 7, 7]|
| Mike|   [3, 9, 1, 6, 2]|   [1, 2, 3, 6, 9]|
+-----+------------------+------------------+

